# Training Q Model
## Load Files

In [1]:
from FileUtils import FileUtils
import matplotlib.pyplot as plt 

In [2]:
data = FileUtils()
data.filter_by_class("cat")

Start to load data
Start to load images: 1000=>2000=>3000=>4000=>5000=>6000=>7000=>8000=>9000=>10000=>11000=>11540!
Start to load annotations: 1000=>2000=>3000=>4000=>5000=>6000=>7000=>8000=>9000=>10000=>11000=>11540!
End


In [4]:
len(data.images)

1084